In [5]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import folium

# 1️⃣ Define the place
place = "Nairobi, Kenya"
admin = ox.geocode_to_gdf(place)



# 2️⃣ Fetch forest polygons from OpenStreetMap
forest = ox.features_from_place(
    place,
    tags={"landuse": "forest", "natural": "wood"}
)

# 3️⃣ Keep only polygon geometries
forest = forest[forest.geometry.type.isin(["Polygon", "MultiPolygon"])]

# 4️⃣ Reproject to UTM zone (meters) for accurate area measurement
forest = forest.to_crs(epsg=32737)  # UTM Zone 37S (Kenya region)

# 5️⃣ Calculate area
forest["area_m2"] = forest.geometry.area
forest["area_ha"] = forest["area_m2"] / 10_000  # convert m² → hectares

# 6️⃣ Group by forest name (if available)
# Many OSM features have a 'name' attribute like 'Karura Forest'
if "name" in forest.columns:
    summary = forest.groupby("name", dropna=False)["area_ha"].sum().reset_index()
else:
    summary = pd.DataFrame(columns=["name", "area_ha"])

# 7️⃣ Compute totals
total_area_ha = forest["area_ha"].sum()
total_area_km2 = total_area_ha / 100

print(f"🌲 Total forest area in Nairobi: {total_area_ha:.2f} ha ({total_area_km2:.2f} km²)\n")
print("📊 Breakdown by forest:")
print(summary.sort_values("area_ha", ascending=False).to_string(index=False))

# 8️⃣ (Optional) Visualize on Folium
forest_display = forest.to_crs(epsg=4326)
m = folium.Map(location=[-1.286389, 36.817223],
               zoom_start=11,
               tiles="cartoDB dark matter",
               control_scale= True)

admin_layer = folium.FeatureGroup(name="Admin Boundary", show=True)

folium.GeoJson(
    admin,
    style_function=lambda feature: {
        "fillColor": "#ffffff",
        "color": "#ffffff",
        "weight": 3,
        "fillOpacity": 0.15
    }
).add_to(admin_layer)
admin_layer.add_to(m)

# Add polygons with hover labels
folium.GeoJson(
    forest_display,
    style_function=lambda x: {"color": "green", "fillOpacity": 0.5},
    tooltip=folium.GeoJsonTooltip(fields=["name", "area_ha"], aliases=["Name", "Area (ha):"])
).add_to(m)

m.save("nairobi_forest_summary.html")
m

🌲 Total forest area in Nairobi: 4310.24 ha (43.10 km²)

📊 Breakdown by forest:
                                   name     area_ha
                                    NaN 2145.994177
                          Karura Forest  681.105316
            Ngong Road Forest Section 4  577.330819
                         Oloolua Forest  411.522338
                         Sigiria Forest  267.607191
            Ngong Road Forest Section 3   65.151167
          Ngong Road Forest - Section 2   54.364718
Giraffe Center Nature Trail (Sanctuary)   45.284129
                      Nairobi Arboretum   32.560624
          Ngong Road Forest - Section 1   20.731447
                Reserved Insect Science    7.266667
                                forest1    1.322209
